In [ ]:
# from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling, Dense, Conv2D, BatchNormalization, Dropout, MaxPooling2D, ReLU, AveragePooling2D, Flatten, ZeroPadding2D, DepthwiseConv2D, SeparableConv2D
from tensorflow.keras import losses, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential, save_model, load_model

from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf

import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
tf.__version__

In [ ]:
image_size = (224, 224)
batch_size = 50
epochs = 2

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

# !ls "/content/drive/MyDrive/Colab Notebooks/Masters"

In [ ]:
# base_dir = '/media/ext_mount/Project/BuildingCNN'
base_dir = '/content/drive/MyDrive/Colab Notebooks/Masters'

raw_dataset_directory = f"{base_dir}/Dataset/GC10-DET/images"
dataset_directory = f'{base_dir}/Dataset/GC10-DET/dataset'

train_set_directory = f'{dataset_directory}/train'
test_set_directory = f'{dataset_directory}/test'

model_directory = f'{base_dir}/models/custom_cnn_600K'

**Preparing Dataset**

In [ ]:
# !unzip "/content/drive/My Drive/Colab Notebooks/Masters/GC10-DET.zip" -d "/content/drive/My Drive/Colab Notebooks/Masters/GC10-DET"
# !pip install split-folders 

In [ ]:
# import splitfolders
# splitfolders.ratio(raw_dataset_directory, output=dataset_directory, seed=555, ratio=(.9, .1), group_prefix=None)

In [ ]:
train_ds_batch = tf.keras.preprocessing.image_dataset_from_directory(train_set_directory,
    validation_split=0.1,
    subset="training",
    seed=555,
    image_size=image_size,
    batch_size=batch_size,
)
train_ds = train_ds_batch.prefetch(buffer_size=32)

In [ ]:
val_ds_batch = tf.keras.preprocessing.image_dataset_from_directory(train_set_directory,
    validation_split=0.1,
    subset="validation",
    seed=555,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = val_ds_batch.prefetch(buffer_size=32)

In [ ]:
test_ds_batch = tf.keras.preprocessing.image_dataset_from_directory(test_set_directory,
    # validation_split=0.1,
    # subset="validation",
    seed=555,
    image_size=image_size,
    batch_size=batch_size,
)
test_ds = test_ds_batch.prefetch(buffer_size=32)

In [ ]:
class_names = train_ds_batch.class_names
class_names

In [ ]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(int(labels[i]))
#         plt.axis("off")

In [ ]:
def show_image(img, fig_size=(4, 4)):
    # show image
    plt.figure(figsize=fig_size)
    plt.imshow(img.astype('uint8'))
    plt.show()

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

## Modelling

#### 600 K WITH DEPTHWISE

In [ ]:
model_type = '600K_With_Depthwise'

In [ ]:
model = Sequential()
model.add(Rescaling(1./255, input_shape=(224, 224, 3)))

model.add(Conv2D(16, (3,3), dilation_rate=1, padding='same', input_shape=(224, 224, 3)))
model.add(ReLU())
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), dilation_rate=1, padding='same', input_shape=(224, 224, 16)))
model.add(ReLU())
model.add(BatchNormalization())
model.add(Dropout(0.03))

# model.add(SeparableConv2D(32, (3,3), dilation_rate=1, padding='same', input_shape=(32, 32, 16)))
model.add(DepthwiseConv2D(kernel_size=(3,3), dilation_rate=1, depth_multiplier=1, padding='same', input_shape=(224,224,32)))
model.add(Conv2D(64, (1,1), dilation_rate=1, input_shape=(224, 224, 32))) # padding='same', 
model.add(ReLU())
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), dilation_rate=1, padding='same', input_shape=(224, 224, 64)))
model.add(ReLU())
model.add(BatchNormalization())
model.add(Dropout(0.03))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(DepthwiseConv2D(kernel_size=(3,3), dilation_rate=1, depth_multiplier=1, padding='same', input_shape=(224,224,64)))
model.add(Conv2D(96, (1,1), dilation_rate=1, input_shape=(224, 224, 64)))
model.add(ReLU())
model.add(BatchNormalization())

model.add(Conv2D(96, (3,3), dilation_rate=2, padding='same', input_shape=(224, 224, 96)))
model.add(ReLU())
model.add(BatchNormalization())
model.add(Dropout(0.07))

model.add(DepthwiseConv2D(kernel_size=(3,3), dilation_rate=1, depth_multiplier=1, padding='same', input_shape=(224,224,96)))
model.add(Conv2D(128, (1,1), dilation_rate=1, input_shape=(224, 224, 96)))
model.add(ReLU())
model.add(BatchNormalization())


model.add(DepthwiseConv2D(kernel_size=(3,3), dilation_rate=1, depth_multiplier=1, padding='same', input_shape=(224,224,128)))
model.add(Conv2D(192, (1,1), dilation_rate=1, input_shape=(224, 224, 128)))
model.add(ReLU())
model.add(BatchNormalization())
model.add(Dropout(0.05))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(DepthwiseConv2D(kernel_size=(3,3), dilation_rate=1, depth_multiplier=1, padding='same', input_shape=(224,224,192)))
model.add(Conv2D(260, (1,1), dilation_rate=1, input_shape=(224, 224, 192)))
model.add(ReLU())
model.add(BatchNormalization())

model.add(DepthwiseConv2D(kernel_size=(3,3), dilation_rate=1, depth_multiplier=1, padding='same', input_shape=(224,224,260)))
model.add(Conv2D(320, (1,1), dilation_rate=1, input_shape=(224, 224, 260)))
model.add(ReLU())
model.add(BatchNormalization())
model.add(Dropout(0.02))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(DepthwiseConv2D(kernel_size=(3,3), dilation_rate=1, depth_multiplier=1, padding='same', input_shape=(224,224,320)))
model.add(Conv2D(370, (1,1), dilation_rate=1, input_shape=(224, 224, 320)))
model.add(ReLU())
model.add(BatchNormalization())

model.add(DepthwiseConv2D(kernel_size=(3,3), dilation_rate=1, depth_multiplier=1, padding='same', input_shape=(224,224,370)))
model.add(Conv2D(415, (1,1), dilation_rate=1, input_shape=(224, 224, 370)))
model.add(ReLU())
model.add(BatchNormalization())

model.add(AveragePooling2D(pool_size=15))# pool_size=4
model.add(Flatten())

model.add(Dense(10))

In [ ]:
# model.build(input_shape=(32,32,3))
model.summary(line_length=100)

In [ ]:
# Prepare model model saving directory.
save_dir = os.path.join(model_directory, 'saved_models')
model_name = 'keras_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [ ]:
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

early_stopper = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=100,
    verbose=1,
    restore_best_weights=True
    )

In [ ]:
callbacks = [checkpoint, lr_reducer, lr_scheduler, early_stopper]

In [ ]:
model.compile(
    # loss=losses.SparseCategoricalCrossentropy(),
    # optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1, callbacks=callbacks)

In [ ]:
scores = model.evaluate(test_ds)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

### Save Final Model

In [ ]:
# Save the model
final_model_filepath = os.path.join(save_dir, f'{model_type}_final_model.h5')
save_model(model, final_model_filepath)

# model = load_model(final_model_filepath)

In [ ]:
final_model_filepath

### Test Predictions

In [ ]:
# test_image = f'{test_set_directory}/silk_spot/img_01_425005700_00191.jpg'
test_image = f'{test_set_directory}/silk_spot/img_03_4406645900_00364.jpg'
# test_image = f'{test_set_directory}/silk_spot/img_03_3436786500_00071.jpg'
# test_image = f'{test_set_directory}/oil_spot/img_03_3402617700_00118.jpg'

In [ ]:
img = tf.keras.utils.load_img(test_image, target_size=image_size)
img_array = tf.keras.utils.img_to_array(img)
# show_image(img_array)

In [ ]:
img_array = img_array/255
img_array = tf.expand_dims(img_array, 0) # Create a batch
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print("{} : {:.2f} %".format(class_names[np.argmax(score)], 100 * np.max(score)))

### Inference Timings

In [ ]:
import time

In [ ]:
# single image inference timings
inference_timings = []

one_batch = iter(test_ds_batch).get_next()

for index, img in enumerate(one_batch[0]):
    
    actual_category = one_batch[1][index]
    start_time = time.time()
    img_array = tf.expand_dims(img, 0) # Create a batch
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    inference_timings.append(time.time()-start_time)
        
#     print('Prediction time: {}'.format(time.time()-start_time))
#     print("Actual: {} \t|\t Predicted: {} : {:.2f} %".format(
#         class_names[actual_category],
#         class_names[np.argmax(score)], 
#         100 * np.max(score)
#     ))
#     break

print('Average inference time: {:.2f} ms'.format(np.array(inference_timings).mean()*1000))

In [ ]:
# batch inference timings

test_batch_id = 3
start_time = time.time()
one_batch = iter(test_ds_batch).get_next()
model.predict(one_batch[0])
end_time = time.time()-start_time

print('Total Prediction time: {:.2f} ms. Per image time: {:.2f} ms'.format(
    end_time*1000, 1000*end_time/batch_size
    ))

### **References**

In [ ]:
# from torchinfo import summary
# import torchvision
# import torch.nn as nn

In [ ]:
# class SimpleModel(composer.models.MosaicClassifier):
#     def __init__(self, num_classes: int):
#         module = nn.Sequential(
#     ################################## 1 ST CONVOLUTIONAL BLOCK #####################################  
#             nn.Conv2d(in_channels = 3,out_channels = 16, dilation  = 1,padding = 1, kernel_size= (3,3)),                     # in 32, out 32, RF 3
#             nn.ReLU(),
#             nn.BatchNorm2d(16),                                                                               #26x26 /8 
#             nn.Conv2d(in_channels = 16,out_channels = 32, dilation  = 1,padding = 1, kernel_size= (3,3)),                     # in 32, out 32, RF 3
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.Dropout(0.03),
#             # nn.MaxPool2d(2, 2),
#             nn.Conv2d(in_channels = 32,out_channels = 32,groups = 32, dilation  = 1,padding = 1,kernel_size= (3,3)),
#             nn.Conv2d(in_channels = 32,out_channels = 64, dilation = 1,padding = 0,kernel_size= (1,1)), # 8, 8, 3  
#             nn.ReLU(),
#             nn.BatchNorm2d(64)  ,          
#             nn.Conv2d(in_channels = 64,out_channels = 64, dilation  = 1,padding = 1, kernel_size= (3,3)),                     
#             nn.ReLU(),
#             nn.BatchNorm2d(64),
#             nn.Dropout(0.03),
#             nn.MaxPool2d(2, 2),                                                                                             # in 32, out 16, RF 3

#             nn.Conv2d(in_channels = 64,out_channels = 64,groups = 64, dilation  = 1,padding = 1,kernel_size= (3,3)),        
#             nn.Conv2d(in_channels = 64,out_channels = 96, dilation = 1,padding = 0,kernel_size= (1,1)), # 8, 8, 3           
#             nn.ReLU(),
#             nn.BatchNorm2d(96),
#             nn.Conv2d(in_channels = 96,out_channels = 96, dilation  = 2,padding = 2, kernel_size= (3,3)),                     
#             nn.ReLU(),
#             nn.BatchNorm2d(96),
#             nn.Dropout(0.07),
#             # nn.MaxPool2d(2, 2),
#             nn.Conv2d(in_channels = 96,out_channels = 96, groups = 96, dilation  = 1,padding = 1,kernel_size= (3,3)),    
#             nn.Conv2d(in_channels = 96,out_channels = 128, dilation  = 1,padding = 0,kernel_size= (1,1)),                  
#             nn.ReLU(),
#             nn.BatchNorm2d(128),
#             nn.Conv2d(in_channels = 128,out_channels = 128, groups = 128, dilation  = 1,padding = 1,kernel_size= (3,3)),    
#             nn.Conv2d(in_channels = 128,out_channels = 192, dilation  = 1,padding = 0,kernel_size= (1,1)),                  
#             nn.ReLU(),
#             nn.BatchNorm2d(192),
#             nn.Dropout(0.05),
#             nn.MaxPool2d(2, 2),                                                                                              # in 16, out 8, RF 3
            
#             nn.Conv2d(in_channels = 192,out_channels = 192, groups = 192, dilation  = 1,padding = 1,kernel_size= (3,3)),    
#             nn.Conv2d(in_channels = 192,out_channels = 260, dilation  = 1,padding = 0,kernel_size= (1,1)),                  
#             nn.ReLU(),
#             nn.BatchNorm2d(260),            
#             nn.Conv2d(in_channels = 260,out_channels = 260, groups = 260, dilation  = 1,padding = 1,kernel_size= (3,3)),    
#             nn.Conv2d(in_channels = 260,out_channels = 320, dilation  = 1,padding = 0,kernel_size= (1,1)),                  
#             nn.ReLU(),
#             nn.BatchNorm2d(320),
#             nn.Dropout(0.02),
#             nn.MaxPool2d(2, 2), 
            
            
#             nn.Conv2d(in_channels = 320,out_channels = 320, groups = 320, dilation  = 1,padding = 1,kernel_size= (3,3)),    
#             nn.Conv2d(in_channels = 320,out_channels = 370, dilation  = 1,padding = 0,kernel_size= (1,1)),                  
#             nn.ReLU(),
#             nn.BatchNorm2d(370),                       
#             # nn.Dropout(0.02),
#              nn.Conv2d(in_channels = 370,out_channels = 370, groups = 370, dilation  = 1,padding = 1,kernel_size= (3,3)),    
#             nn.Conv2d(in_channels = 370,out_channels = 415, dilation  = 1,padding = 0,kernel_size= (1,1)),                  
#             nn.ReLU(),
#             nn.BatchNorm2d(415),

#             nn.AvgPool2d(kernel_size=4),                                                                      #1x1/15
#             nn.Flatten(),
#             nn.Linear(415,10)
#         )
#         self.num_classes = num_classes
#         super().__init__(module=module)

In [ ]:
# summary(m,input_size = (1,3,32,32))